## SETUP 

In [1]:
#import libraries
import pandas as pd
import sqlite3

In [4]:
#df = pd.DataFrame(#data here :/)

df = pd.DataFrame({'name': ['Juan', 'Victoria', 'Mary'], \
'age': [23, 34, 43], 'city': ['Miami', 'Buenos Aries', 'Santiago']})


In [5]:
df

,name,age,city
0,Juan,23,Miami
1,Victoria,34,Buenos Aries
2,Mary,43,Santiago


In [7]:
#We will sqlite3 library and create a connection
cnn = sqlite3.connect('jupyter_sql_tutorial.db')

In [11]:
df.to_sql('people', cnn)

In [12]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [15]:
%sql sqlite:///jupyter_sql_tutorial.db

In [17]:
%%sql
SELECT *
FROM people

 * sqlite:///jupyter_sql_tutorial.db
Done.


index,name,age,city
0,Juan,23,Miami
1,Victoria,34,Buenos Aries
2,Mary,43,Santiago


In [18]:
%%sql
SELECT count(*)
FROM people

 * sqlite:///jupyter_sql_tutorial.db
Done.


count(*)
3


In [19]:
%%sql
SELECT sum(age) as 'age_sum'
FROM people

 * sqlite:///jupyter_sql_tutorial.db
Done.


age_sum
100


## Parameters walkthrough

In [20]:
#create dummy dataframe
df = pd.DataFrame({'transaction_id': ['9', '8', '7', '6', '5', '4', '3'], \
                  'user_id': ['rafa', 'roy', 'kenny', 'brendan', 'jurgen', 'roy', 'roy'],\
                  'transaction_date': ['2021-12-21', '2020-12-21', '2019-12-21',\
                                       '2018-11-21', '2017-10-21', '2019-03-02', '2010-01-01'],\
                  'amount': ['10', '15', '20', '24', '25', '31', '42']})

In [21]:
df

,transaction_id,user_id,transaction_date,amount
0,9,rafa,2021-12-21,10
1,8,roy,2020-12-21,15
2,7,kenny,2019-12-21,20
3,6,brendan,2018-11-21,24
4,5,jurgen,2017-10-21,25
5,4,roy,2019-03-02,31
6,3,roy,2010-01-01,42


In [22]:
#We will sqlite3 library and create a connection
cnn = sqlite3.connect('dummy.db')

In [29]:
df.to_sql('managers1', cnn)

In [30]:
%reload_ext sql

In [31]:
%sql sqlite:///dummy.db

In [32]:
%%sql
SELECT *
FROM managers1

 * sqlite:///dummy.db
Done.


index,transaction_id,user_id,transaction_date,amount
0,9,rafa,2021-12-21,10
1,8,roy,2020-12-21,15
2,7,kenny,2019-12-21,20
3,6,brendan,2018-11-21,24
4,5,jurgen,2017-10-21,25
5,4,roy,2019-03-02,31
6,3,roy,2010-01-01,42


In [34]:
%%sql
SELECT sum(amount) as 'spend_sum'
FROM managers1

 * sqlite:///dummy.db
Done.


spend_sum
167


In [35]:
%%sql
SELECT 
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
FROM
    managers1
WHERE
    user_id = 'roy'
    and transaction_date = '2019-03-02'

 * sqlite:///dummy.db
Done.


user_id,num_transactions,total_amount
roy,1,31


## For real this time

ovechal01, ovi8

1228 * 11,


In [1]:
#import libraries
import pandas as pd
import sqlite3

In [2]:
stat_handle = input('stat handle:')
stat_sheet = f'./data/{stat_handle}_stats.csv'
stat_df = pd.read_csv(stat_sheet, sep= '\t', header= 0, index_col= None)

stat handle:ovechal01


In [3]:
stat_df

,date,goals,assists,+/-,pim,shots,shot%,TOI,hits,blocks,faceoff %
0,2013-01-19 19:00:00-05:00,0,0,0,2,4,0.0,23:35,2,0,NaN
1,2013-01-22 19:00:00-05:00,0,1,0,0,4,0.0,23:59,4,0,NaN
2,2013-01-24 19:00:00-05:00,0,0,-1,2,3,0.0,22:26,1,0,NaN
3,2013-01-25 19:00:00-05:00,0,0,0,0,2,0.0,22:29,1,0,NaN
4,2013-01-27 15:00:00-05:00,1,0,-1,0,5,20.0,18:29,3,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
675,2021-12-29 19:00:00-05:00,0,1,1,0,4,0.0,26:01,0,0,NaN
676,2021-12-31 19:30:00-05:00,2,0,2,0,7,28.6,19:54,1,0,NaN
677,2022-01-02 15:00:00-05:00,0,0,-2,0,3,0.0,22:56,4,0,100.0
678,2022-01-07 20:00:00-05:00,0,0,-3,0,3,0.0,20:55,1,0,NaN


In [4]:
stat_handle = input('twitter handle:')
stat_sheet = f'./data/{stat_handle}.csv'
twitter_df = pd.read_csv(stat_sheet, sep= '\t', header= 0, index_col= None)

twitter handle:ovi8


In [5]:
twitter_df

,date time,display name,content
0,2021-12-02 18:19:51-05:00,Alex Ovechkin,The 2022 #CapsCanineCalendar is Ovi and Blake ...
1,2021-11-20 20:44:54-05:00,Alex Ovechkin,🏆 You took the biggest step!!! Congrats @WashS...
2,2021-11-16 04:04:50-05:00,Alex Ovechkin,Great to be part of the first @blockassetco Le...
3,2021-11-04 13:16:26-04:00,Alex Ovechkin,Good luck and see you guys soon in DC!! https:...
4,2021-10-28 22:15:31-04:00,Alex Ovechkin,Very proud to be a @Blockassetco Legend with @...
...,...,...,...
620,2009-01-21 14:51:20-05:00,Alex Ovechkin,At DC airport with tons of obama fans. Taking ...
621,2009-01-20 03:40:04-05:00,Alex Ovechkin,Lots of questions about my trick shot plans - ...
622,2009-01-20 01:35:55-05:00,Alex Ovechkin,Only thing better than scoring goals is winnin...
623,2009-01-19 04:31:30-05:00,Alex Ovechkin,Big win last night on a sick shot by Semin. No...


In [29]:
game_list = []
gm_dates = stat_df.iloc[:,0]
for gm in gm_dates:
    try:
        gm = pd.to_datetime(gm)
        #print(gm)
        #gm = gm - timedelta(days=1)
        #print(gm, '\n\n')
        game_list.append(gm)
    except:
        continue
    
print(len(game_list))

2013-01-19 19:00:00-05:00
2013-01-18 19:00:00-05:00 


2013-01-22 19:00:00-05:00
2013-01-21 19:00:00-05:00 


2013-01-24 19:00:00-05:00
2013-01-23 19:00:00-05:00 


2013-01-25 19:00:00-05:00
2013-01-24 19:00:00-05:00 


2013-01-27 15:00:00-05:00
2013-01-26 15:00:00-05:00 


2013-01-29 19:30:00-05:00
2013-01-28 19:30:00-05:00 


2013-01-31 19:00:00-05:00
2013-01-30 19:00:00-05:00 


2013-02-01 19:00:00-05:00
2013-01-31 19:00:00-05:00 


2013-02-04 00:30:00-05:00
2013-02-03 00:30:00-05:00 


2013-02-05 19:00:00-05:00
2013-02-04 19:00:00-05:00 


2013-02-07 19:00:00-05:00
2013-02-06 19:00:00-05:00 


2013-02-09 19:00:00-05:00
2013-02-08 19:00:00-05:00 


2013-02-12 19:30:00-05:00
2013-02-11 19:30:00-05:00 


2013-02-14 19:30:00-05:00
2013-02-13 19:30:00-05:00 


2013-02-17 18:00:00-05:00
2013-02-16 18:00:00-05:00 


2013-02-21 19:00:00-05:00
2013-02-20 19:00:00-05:00 


2013-02-24 00:00:00-05:00
2013-02-23 00:00:00-05:00 


2013-02-26 19:00:00-05:00
2013-02-25 19:00:00-05:00 


2013-02-27

2016-02-10 20:00:00-05:00 


2016-02-13 20:00:00-05:00
2016-02-12 20:00:00-05:00 


2016-02-16 19:00:00-05:00
2016-02-15 19:00:00-05:00 


2016-02-18 19:00:00-05:00
2016-02-17 19:00:00-05:00 


2016-02-20 19:00:00-05:00
2016-02-19 19:00:00-05:00 


2016-02-22 19:00:00-05:00
2016-02-21 19:00:00-05:00 


2016-02-24 19:00:00-05:00
2016-02-23 19:00:00-05:00 


2016-02-26 19:00:00-05:00
2016-02-25 19:00:00-05:00 


2016-02-29 00:30:00-05:00
2016-02-28 00:30:00-05:00 


2016-03-01 19:30:00-05:00
2016-02-29 19:30:00-05:00 


2016-03-02 19:00:00-05:00
2016-03-01 19:00:00-05:00 


2016-03-04 19:00:00-05:00
2016-03-03 19:00:00-05:00 


2016-03-05 19:00:00-05:00
2016-03-04 19:00:00-05:00 


2016-03-07 22:00:00-05:00
2016-03-06 22:00:00-05:00 


2016-03-09 22:30:00-05:00
2016-03-08 22:30:00-05:00 


2016-03-12 22:30:00-05:00
2016-03-11 22:30:00-05:00 


2016-03-15 19:00:00-04:00
2016-03-14 19:00:00-04:00 


2016-03-18 19:00:00-04:00
2016-03-17 19:00:00-04:00 


2016-03-20 18:00:00-04:00
2016-03-19

2019-11-12 19:30:00-05:00 


2019-11-15 19:00:00-05:00
2019-11-14 19:00:00-05:00 


2019-11-16 19:00:00-05:00
2019-11-15 19:00:00-05:00 


2019-11-18 19:00:00-05:00
2019-11-17 19:00:00-05:00 


2019-11-20 20:00:00-05:00
2019-11-19 20:00:00-05:00 


2019-11-24 00:30:00-05:00
2019-11-23 00:30:00-05:00 


2019-11-27 19:00:00-05:00
2019-11-26 19:00:00-05:00 


2019-11-29 17:00:00-05:00
2019-11-28 17:00:00-05:00 


2019-11-30 19:00:00-05:00
2019-11-29 19:00:00-05:00 


2019-12-03 22:30:00-05:00
2019-12-02 22:30:00-05:00 


2019-12-04 22:00:00-05:00
2019-12-03 22:00:00-05:00 


2019-12-06 22:00:00-05:00
2019-12-05 22:00:00-05:00 


2019-12-09 19:00:00-05:00
2019-12-08 19:00:00-05:00 


2019-12-11 19:00:00-05:00
2019-12-10 19:00:00-05:00 


2019-12-14 19:00:00-05:00
2019-12-13 19:00:00-05:00 


2019-12-16 19:00:00-05:00
2019-12-15 19:00:00-05:00 


2019-12-20 19:00:00-05:00
2019-12-19 19:00:00-05:00 


2019-12-21 19:00:00-05:00
2019-12-20 19:00:00-05:00 


2019-12-23 19:00:00-05:00
2019-12-22

Not catching data currently, date ranges need to be defined 

In [19]:
from datetime import timedelta

data_points = []
post_game = 0
tweet_dates = twitter_df.iloc[:,0]
#tweet_times = twitter_df.iloc[:,1]
for dt in tweet_dates:
    dt = pd.to_datetime(dt)
    #Use timedates to create if statements
    #Try within 24 hours of puckdrop, also from midnight on gameday
    if dt in game_list:
        gm_time = pd.to_datetime(gm_time)
        gm = pd.to_datetime(gm)
        if gm < gm_time:
            data_points.append(gm)
        else:
            post_game += 1


print(f'{len(data_points)} good points of data, {post_game} postgame tweets')

0 good points of data, 0 postgame tweets


In [9]:
for d in data_points:
    print(d)

Maybe need to include puck drop times? Would require altering stat scraper, and likely slowing it down signifigantly, to the magnitute of rendering an entire new webpage in beautfiul soup per row in the dataset. For Ovechkin this would include 1228 pages! Maybe I should use selenium as an alternative. However at least for Ovechkin, no gameday tweets were scheduled for 7pm EST. BRB Looking at tweet timestamps

In [9]:
for point in data_points:
    print(point)